In [ ]:
#importing prep_data file, jupyter notebook style
%run Prep_data_All_Patients.ipynb

In [ ]:
supDir = '/Users/elikond/Downloads/surprisal_analysis/'
clusterDir = '/Users/elikond/Downloads/clusters/'

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
import shap

In [ ]:
def xgboost(supDir, clusterDir, mesenDir, patientID):
    merged_df, X_data, y_data, barcode_len, sigSubpopsDF = final(supDir, clusterDir, mesenDir, patientID)
        
    X_data_arr = X_data.to_numpy()
    y_data_arr = y_data.to_numpy()
    
    X_train, X_test, y_train, y_test = train_test_split(X_data_arr, y_data_arr, test_size=0.33, random_state=42)
    
    print(X_train)
    #print(y_data_arr)
    # create XGBoost instance with default hyper-parameters
    xgb_estimator = xgb.XGBClassifier(objective='binary:logistic')

    # create MultiOutputClassifier instance with XGBoost model inside
    multilabel_model = MultiOutputClassifier(xgb_estimator)

    # fit the model
    multilabel_model.fit(X_train, y_train)
    
    y_predict = multilabel_model.predict(X_test)

    for i in range(barcode_len):
        precision, recall, thresholds = precision_recall_curve(y_test[:,i], y_predict[:,i])
        auc_precision_recall = auc(recall, precision)
        print(auc_precision_recall)
        
    explainer = shap.Explainer(multilabel_model)
    shap_values = explainer.shap_values(X_train)
    np.abs(shap_values.sum(1) + explainer.expected_value - pred).max()
        
    return multilabel_model

In [ ]:
xgboost(supDir, clusterDir, mesenDir, 'gb13')

In [ ]:
def cross_patient_xgboost(supDir, clusterDir, mesenDir, train_patientID, test_patientID):
    mergedTrain_DF, XTrain, yTrain, barcodeTrain_len, Train_sigSubpopsDF = final(supDir, clusterDir, mesenDir, train_patientID)
    mergedTest_DF, XTest, yTest, barcodeTest_len, Test_sigSubpopsDF = final(supDir, clusterDir, mesenDir, test_patientID)
    
    shared_col = list(XTrain.columns & XTest.columns)
    
    XTrain_shared = XTrain[shared_col]
    XTrain_arr = XTrain_shared.to_numpy()
    yTrain_arr = yTrain.to_numpy()

    XTest_shared = XTest[shared_col]
    XTest_arr = XTest_shared.to_numpy()
    yTest_arr = yTest.to_numpy()

    # create XGBoost instance with default hyper-parameters
    xgb_estimator = xgb.XGBClassifier(objective='binary:logistic')

    # create MultiOutputClassifier instance with XGBoost model inside
    multilabel_model = MultiOutputClassifier(xgb_estimator)

    # fit the model
    multilabel_model.fit(XTrain_arr, yTrain_arr)
    
    y_predict = multilabel_model.predict(XTest_arr)

    for i in range(barcodeTest_len):
        precision, recall, thresholds = precision_recall_curve(yTest_arr[:,i], y_predict[:,i])
        auc_precision_recall = auc(recall, precision)
        print(auc_precision_recall)

In [ ]:
cross_patient_xgboost(supDir, clusterDir, _, 'gb13', 'gb9')